In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Replace with your actual file path
file_path = '/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/Health_and_Personal_Care.jsonl'

# Load the .jsonl file into a pandas DataFrame
df = pd.read_json(file_path, lines=True)

# Display the first 5 rows of the DataFrame
print("DataFrame Loaded Successfully!")
df.head()


DataFrame Loaded Successfully!


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True


In [ ]:
# number of reviews
print(len(df))

print(list(df.columns))

494121
['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase']


In [ ]:
# count of all ratings to check if there's imbalance
df['rating'].value_counts()

,count
rating,
5,301713
1,69564
4,57000
3,36949
2,28895


There seems to be a class imbalance. 5 star reviews seem to dominate a lot.

In [ ]:
df.isnull().sum()

,0
rating,0
title,0
text,0
images,0
asin,0
parent_asin,0
user_id,0
timestamp,0
helpful_vote,0
verified_purchase,0


There doesn't seem to be any null values.

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC                           # <-- Import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

# Preprocessing

In [ ]:
df.dropna(subset=['text', 'rating'], inplace=True)
print(f"\nDataFrame shape after dropping rows with missing text/rating: {df.shape}")

# Combine 'title' and 'text'
# Fill missing titles with an empty string BEFORE concatenation
df['title'] = df['title'].fillna('')
df['review_full'] = df['title'] + ' ' + df['text']

# Text Cleaning Setup
nltk.download('punkt_tab', quiet=True) # Download the specific resource needed
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. Lowercasing
    text = text.lower()
    # 2. Remove HTML tags (if any)
    text = re.sub(r'<.*?>', '', text)
    # 3. Remove punctuation and special characters (keeping only letters and whitespace)
    text = re.sub(r'[^a-z\s]', '', text)
    # 4. Tokenization
    tokens = word_tokenize(text)
    # 5. Remove Stop Words and Lemmatize
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 1] # Keep words with length > 1
    # 6. Join back into string
    return ' '.join(cleaned_tokens)

print("\nStarting text cleaning (this may take a while)...")
# Apply cleaning function - Monitor progress if needed for large datasets
# Consider df['review_full'].parallel_apply(clean_text) using libraries like pandarallel if speed is critical
df['review_cleaned'] = df['review_full'].apply(clean_text)
print("Text cleaning completed.")

# Drop rows where cleaning might have resulted in empty strings
df.dropna(subset=['review_cleaned'], inplace=True)
df = df[df['review_cleaned'].str.strip() != ''] # Ensure no empty strings after cleaning
print(f"DataFrame shape after cleaning and removing empty reviews: {df.shape}")


DataFrame shape after dropping rows with missing text/rating: (494121, 11)

Starting text cleaning (this may take a while)...
Text cleaning completed.
DataFrame shape after cleaning and removing empty reviews: (493820, 12)


In [ ]:
df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,review_full,review_cleaned
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True,12 mg is 12 on the periodic table people! Mg f...,mg periodic table people mg magnesium review c...
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True,Save the lanet using less plastic. Love these ...,save lanet using less plastic love easy multit...
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True,Fantastic I have been suffering a couple month...,fantastic suffering couple month heel pain pla...
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True,It holds the water and makes bubbles. That's ...,hold water make bubble thats bought cheap want...
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True,Not for me Didn't do a thing for me. Not sayin...,didnt thing saying dont


Saving the processed data frame in a file (pickle, parquet and csv) as a checkpoints.

In [ ]:
# saving checkpoint
df.to_pickle('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.pkl')

NameError: name 'df' is not defined

In [ ]:
df.to_parquet('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.parquet')

In [ ]:
df.to_csv('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.csv', index=False)

#Load from this checkpoint if saved already

In [7]:
import pandas as pd

# loading checkpoint
df = pd.read_pickle('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.pkl')

# --- 3. Feature Engineering (TF-IDF) ---

In [8]:
# Define Features (X) and Target (y)
X = df['review_cleaned']
y = df['rating']

# Initialize TF-IDF Vectorizer
# You can experiment with parameters like max_features, ngram_range etc. later
# max_features=5000 limits the vocabulary size, can speed up training and reduce memory
# tfidf_vectorizer = TfidfVectorizer(max_features=10000, min_df=5, max_df=0.7) # Example parameters
tfidf_vectorizer = TfidfVectorizer() # Example parameters
print("\nInitializing TF-IDF Vectorizer...")




Initializing TF-IDF Vectorizer...


# --- 4. Data Splitting ---

In [9]:
print("Splitting data into Training and Testing sets (80/20)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 20% for testing
    random_state=42,  # for reproducibility
    stratify=y        # IMPORTANT: keep class distribution same in train/test due to imbalance
)
print(f"Training set size: {len(X_train)} samples")
print(f"Testing set size: {len(X_test)} samples")


Splitting data into Training and Testing sets (80/20)...
Training set size: 395056 samples
Testing set size: 98764 samples


In [10]:
# --- CHECK TOTAL FEATURES ---
# Initialize a temporary vectorizer just to check vocab size
temp_vectorizer = TfidfVectorizer()
print("\nFitting temporary vectorizer to training data to find total features...")
temp_vectorizer.fit(X_train)
total_features = len(temp_vectorizer.vocabulary_)
print(f"Total unique features (terms) found in training data: {total_features}")
# You can now discard temp_vectorizer
del temp_vectorizer
# --- END OF CHECK ---


Fitting temporary vectorizer to training data to find total features...
Total unique features (terms) found in training data: 149434


# --- 5. Apply TF-IDF & Model Training ---

In [ ]:
# Fit TF-IDF on Training data and Transform Training data
print("\nApplying TF-IDF to training data...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
print(f"TF-IDF applied to training data. Shape: {X_train_tfidf.shape}")

# Transform Test data using the *same* fitted vectorizer
print("Applying TF-IDF to testing data...")
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print(f"TF-IDF applied to testing data. Shape: {X_test_tfidf.shape}")

# Train Support Vector Classifier (SVC) with Class Weighting
print("\nTraining SVC model with class_weight='balanced'...")
print("WARNING: SVC with the default 'rbf' kernel can be VERY slow on large datasets!")
start_time = time.time()

# Initialize the SVC classifier
svm_model = SVC(
    class_weight='balanced', # Handles imbalance
    random_state=42,         # For reproducibility
    # kernel='rbf',          # Default kernel, potentially slow
    # C=1.0,                 # Default regularization parameter
    # gamma='scale'          # Default gamma value
    # probability=False      # Default, faster. Set to True if you need predict_proba later.
)

# Fit the model on the TF-IDF transformed training data
svm_model.fit(X_train_tfidf, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"Model training completed in {training_time:.2f} seconds.")


Applying TF-IDF to training data...
TF-IDF applied to training data. Shape: (395056, 100)
Applying TF-IDF to testing data...
TF-IDF applied to testing data. Shape: (98764, 100)

Training SVC model with class_weight='balanced'...


# --- 6. Model Evaluation ---

In [ ]:
print("\nEvaluating model performance on the Test set...")

# Predict on the test set using the trained SVM model
y_pred = svm_model.predict(X_test_tfidf) # Use the svm_model here

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nOverall Accuracy: {accuracy:.4f}")

# Detailed Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=[str(i) for i in sorted(y.unique())], zero_division=0))

# Confusion Matrix
print("\nConfusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                   index=[f'Actual_{i}' for i in sorted(y.unique())],
                   columns=[f'Predicted_{i}' for i in sorted(y.unique())]))

print("\n--- Script Finished ---")

Linear SVC

In [11]:
from sklearn.svm import LinearSVC

# Fit TF-IDF on Training data and Transform Training data
print("\nApplying TF-IDF to training data...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
print(f"TF-IDF applied to training data. Shape: {X_train_tfidf.shape}")

# Transform Test data using the *same* fitted vectorizer
print("Applying TF-IDF to testing data...")
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print(f"TF-IDF applied to testing data. Shape: {X_test_tfidf.shape}")

print("\nTraining LinearSVC model with class_weight='balanced'...")
start_time = time.time()

# Initialize the LinearSVC classifier
# Note: dual=False is often recommended when n_samples > n_features
# Increased max_iter might be needed for convergence
linear_svm_model = LinearSVC(
    class_weight='balanced',
    random_state=42,
    dual=False, # Can be faster when n_samples > n_features
    max_iter=2000 # Increased from default 1000
)

# Fit the model
linear_svm_model.fit(X_train_tfidf, y_train)

end_time = time.time()
training_time = end_time - start_time
print(f"LinearSVC Model training completed in {training_time:.2f} seconds.")


Applying TF-IDF to training data...
TF-IDF applied to training data. Shape: (395056, 149434)
Applying TF-IDF to testing data...
TF-IDF applied to testing data. Shape: (98764, 149434)

Training LinearSVC model with class_weight='balanced'...
LinearSVC Model training completed in 163.56 seconds.


In [12]:
print("\nEvaluating LinearSVC model performance on the Test set...")
y_pred_linear = linear_svm_model.predict(X_test_tfidf)
accuracy_linear = accuracy_score(y_test, y_pred_linear)
print(f"\nLinearSVC Overall Accuracy: {accuracy_linear:.4f}")
print("\nLinearSVC Classification Report:")
print(classification_report(y_test, y_pred_linear, target_names=[str(i) for i in sorted(y.unique())], zero_division=0))
print("\nLinearSVC Confusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred_linear),
                   index=[f'Actual_{i}' for i in sorted(y.unique())],
                   columns=[f'Predicted_{i}' for i in sorted(y.unique())]))
print("\n--- LinearSVC Evaluation Finished ---")


Evaluating LinearSVC model performance on the Test set...

LinearSVC Overall Accuracy: 0.7339

LinearSVC Classification Report:
              precision    recall  f1-score   support

           1       0.67      0.75      0.70     13909
           2       0.30      0.32      0.31      5779
           3       0.37      0.39      0.38      7387
           4       0.48      0.36      0.41     11395
           5       0.88      0.88      0.88     60294

    accuracy                           0.73     98764
   macro avg       0.54      0.54      0.54     98764
weighted avg       0.73      0.73      0.73     98764


LinearSVC Confusion Matrix:
          Predicted_1  Predicted_2  Predicted_3  Predicted_4  Predicted_5
Actual_1        10388         1534          880          307          800
Actual_2         2063         1833         1006          320          557
Actual_3         1235         1064         2883          985         1220
Actual_4          575          625         1392         4